In [1]:
##########################################
import numpy as np                       # NumPy pour le traitement de tableaux et matrices
import cv2                               # OpenCV pour la vision par ordinateur
from ultralytics import YOLO             # Import de la classe YOLO du module Ultralytics
import os                                # Module pour interagir avec le système d'exploitation
import time                              # Module pour mesurer le temps d'exécution
from PIL import Image   
import matplotlib.pyplot  as plt 
import easyocr
import csv
reader = easyocr.Reader(['fr','en']) # this needs to run only once to load the model into memory                    

In [2]:
#  Chargement des modèles YOLO
##############################################
model_immat = YOLO("yolov8n.pt")

In [3]:
results = model_immat(["plate-1/test/images/plate-153-_jpg.rf.434f34d5b883f43807ae1b4880788130.jpg"])


0: 800x800 1 plate, 89.7ms
Speed: 6.4ms preprocess, 89.7ms inference, 462.6ms postprocess per image at shape (1, 3, 800, 800)


In [4]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'plate'}
 orig_img: array([[[117, 132, 165],
         [140, 155, 188],
         [197, 208, 240],
         ...,
         [128, 186, 205],
         [149, 203, 220],
         [139, 193, 210]],
 
        [[118, 133, 166],
         [140, 155, 188],
         [196, 207, 239],
         ...,
         [124, 182, 201],
         [148, 202, 219],
         [137, 191, 208]],
 
        [[123, 138, 171],
         [145, 160, 193],
         [203, 214, 246],
         ...,
         [117, 175, 194],
         [146, 200, 217],
         [135, 189, 206]],
 
        ...,
 
        [[120, 170, 212],
         [124, 174, 216],
         [118, 168, 210],
         ...,
         [184, 205, 220],
         [201, 222, 237],
         [207, 228, 243]],
 
        [[119, 170, 210],
         [118, 169, 209],
         [121, 172, 212],
         ...,
         [181, 204, 219],
    

In [5]:
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    # im.show()  # show image
    x1, y1, x2, y2 = r.boxes.xyxy[0]
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convert to int values
    im= np.array(im)
    
    print(im.shape) # Print image shape
    cv2.imshow("original", im)
    
    # Cropping an image
    cropped_image = im[y1:y2, x1:x2]
    
    # Display cropped image
    cv2.imshow("cropped", cropped_image)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

(640, 640, 3)


# Vidéo

In [12]:

video_path = "./videos/5579760-hd_1920_1080_30fps.mp4"


from PIL import Image



# Initialize video capture
cap = cv2.VideoCapture(video_path)
liste_pred =[]
output_texts = ""
h_to_add,w_to_add = 300,200
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # If there's no frame to capture, break out of the loop
    if not ret:
        break
    frame_copy = frame.copy()
    cv2.putText(frame_copy,output_texts,(20, 20),fontFace=1,fontScale=1,color=(189,0,75),thickness=2)
    # Perform detection on the frame
    results = model_immat(frame, stream= True, line_width=1, show_conf=False, save=True)
    zone = np.array([[150+h_to_add, 500+w_to_add], [600+h_to_add, 400+w_to_add],[610+h_to_add, 402+w_to_add], [160+h_to_add, 502+w_to_add]])
    zone = zone.reshape((-1, 1, 2))
    cv2.polylines(frame_copy,[zone],color=(0,0,0),isClosed=True)
    for r in results:
        boxes = r.boxes
        names = r.names                                   #classes names list
        confs = r.boxes.conf.float().cpu().tolist()  
        for box , conf in zip(boxes, confs):
            if conf > 0.65:
                label = str(names[0] + " "+ str(conf))

                x1, y1, x2, y2 = box.xyxy[0].int().tolist()  # convert to int values and extract box coordinates
                immat_centre = (int((x1 + x2)/2),int((y1+y2)/2))
                cropped_image = frame[y1:y2, x1:x2]  # Crop the frame using the box coordinates
                # Display original frame
                result = cv2.pointPolygonTest(zone, immat_centre, False)
                if result == 1 :
                    
                    output_texts = reader.readtext(cropped_image)
                    output_texts = str(output_texts[0][1])
                    


                
                cv2.putText(frame_copy,label,(x1, y1),fontFace=1,fontScale=1,color=(189,0,75),thickness=2)
                cv2.circle(frame_copy,immat_centre,2,color=(0,0,255))


                # Display cropped image
                cv2.imshow("Cropped", cropped_image)
            

    # Check if 'q' is pressed to exit
    cv2.imshow("Original", frame_copy)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
cap.release()
cv2.destroyAllWindows()



0: 480x800 (no detections), 57.9ms
Speed: 4.2ms preprocess, 57.9ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 800)
Results saved to runs/detect/predict8

0: 480x800 (no detections), 57.9ms
Speed: 4.5ms preprocess, 57.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)
Results saved to runs/detect/predict8

0: 480x800 (no detections), 57.9ms
Speed: 3.9ms preprocess, 57.9ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 800)
Results saved to runs/detect/predict8

0: 480x800 (no detections), 57.9ms
Speed: 5.4ms preprocess, 57.9ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 800)
Results saved to runs/detect/predict8

0: 480x800 (no detections), 57.7ms
Speed: 3.2ms preprocess, 57.7ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 800)
Results saved to runs/detect/predict8

0: 480x800 (no detections), 57.9ms
Speed: 6.3ms preprocess, 57.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 800)
Results save